In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import logging
from sys import getsizeof

In [ ]:
#def aki_staging(site):
site='MCRI'
if True:
    ext = {'MCW':'dsv', 'UofU':'txt', 'UTSW':'dsv'}
    sep = {'MCW':'|',   'UofU':'|',   'UTSW':'|'}    
    
    print('Running staging on site '+site, flush = True)            
    home_dir = '/home/hchan2/AKI/data2022/'
    if site in ext.keys():       
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB_SCR.'+ext[site],sep=sep[site])
        labcsv.columns = [y.upper() for y in labcsv.columns]        
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.'+ext[site],sep=sep[site])        
        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]                
    elif site =='UPITT':
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB_SCR.csv', encoding='windows-1252')    
        labcsv.columns = [y.upper() for y in labcsv.columns]            
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv')    
        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]        
    else:
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB_SCR.csv',na_values=['NI', '(null)'])    
        labcsv.columns = [y.upper() for y in labcsv.columns]        
        labcsv.columns = [x.replace('"+PD.DATE_SHIFT"', '') for x in labcsv.columns]            
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv',na_values=['NI', '(null)'])    
#        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv', header=None)    
#        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]   
#        onsetcsv.columns = [x.replace('"+PD.DATE_SHIFT"', '') for x in onsetcsv.columns]    

    labcsv = labcsv[labcsv['RESULT_NUM']>0]
    labcsv = labcsv.drop('LAB_RESULT_CM_ID',axis=1).drop_duplicates()
    labcsvX = labcsv.copy()

In [ ]:
onsetcsv.columns = ['PATID', 'ENCOUNTERID', 'ADMIT_DATE', 'DISCHARGE_DATE',
       'SERUM_CREAT_BASE', 'NONAKI_ANCHOR', 'NONAKI_SINCE_ADMIT',
       'NON_AKI_SCR', 'NON_AKI_INC', 'AKI1_ONSET', 'AKI1_SINCE_ADMIT',
       'AKI1_SCR', 'AKI1_INC', 'AKI2_ONSET', 'AKI2_SINCE_ADMIT', 'AKI2_SCR',
       'AKI2_INC', 'BCCOVID', 'AKI3_ONSET', 'AKI3_SINCE_ADMIT', 'AKI3_SCR',
       'AKI3_INC']
onsetcsv.head(2)

In [ ]:
#To datetime
def convert_daytime(df, dateformat='%d-%b-%y', timeformat='%H:%M'):
    for col in df.columns:
        if 'DATE' in col and col.replace('DATE', 'TIME') in df.columns: 
            print(col)
            df[col.replace('DATE', 'DATE_TIME')] = pd.to_datetime(df[col].str.split(' ').str[0]+' '+df[col.replace('DATE', 'TIME')], format=dateformat+' '+timeformat)
        elif 'DATE' in col:
            df.loc[:,col] = pd.to_datetime(df[col], format=dateformat)
    return df

In [ ]:
dateformat='%Y-%m-%d' #UNMC
#onsetcsv = convert_daytime(onsetcsv, dateformat=dateformat)
#labcsv = convert_daytime(labcsv, dateformat=dateformat)
#labcsv = labcsv.drop(['ADMIT_DATE', 'OLD_DAY_SINCE_ADMIT'],axis=1)

In [ ]:
onset_merge = onsetcsv[['ENCOUNTERID', 'ADMIT_DATE']].copy()
onset_merge.columns = ['ONSETS_ENCOUNTERID', 'ADMIT_DATE']
labcsv_merge = labcsv.merge(onset_merge, left_on='ONSETS_ENCOUNTERID', right_on='ONSETS_ENCOUNTERID', how='left')
labcsv_merge = labcsv_merge[labcsv_merge['ADMIT_DATE'].notnull()]
labcsv_merge['NEW_DAY_SINCE_ADMIT'] = (labcsv_merge['SPECIMEN_DATE_TIME']-labcsv_merge['ADMIT_DATE']).dt.total_seconds()/3600/24

In [ ]:
labcsv = labcsv_merge.copy()
labcsv['OLD_DAY_SINCE_ADMIT'] = labcsv['DAYS_SINCE_ADMIT']
labcsv['DAYS_SINCE_ADMIT'] = labcsv['NEW_DAY_SINCE_ADMIT']

In [ ]:
scravg = labcsv[['PATID','ONSETS_ENCOUNTERID','ENCOUNTERID','LAB_ORDER_DATE','SPECIMEN_DATE_TIME', 'RESULT_DATE', 'RESULT_TIME', 'RESULT_NUM', 'DAYS_SINCE_ADMIT']].groupby(['PATID','ONSETS_ENCOUNTERID','ENCOUNTERID','LAB_ORDER_DATE','SPECIMEN_DATE_TIME', 'RESULT_DATE', 'RESULT_TIME', 'DAYS_SINCE_ADMIT']).mean().reset_index()
scravg = scravg[scravg['DAYS_SINCE_ADMIT']>=0]
con03_base = scravg[['PATID', 'ONSETS_ENCOUNTERID', 'RESULT_NUM', 'SPECIMEN_DATE_TIME', 'DAYS_SINCE_ADMIT']].rename(columns={'SPECIMEN_DATE_TIME':'SPECIMEN_DATE_TIME_BASE', 'RESULT_NUM':'RESULT_NUM_BASE', 'DAYS_SINCE_ADMIT':'DAYS_SINCE_ADMIT_BASE'})
con03_base_rn1 = con03_base.loc[con03_base[['PATID', 'ONSETS_ENCOUNTERID', 'DAYS_SINCE_ADMIT_BASE']].groupby('ONSETS_ENCOUNTERID').idxmin().reset_index()['DAYS_SINCE_ADMIT_BASE']]

In [ ]:
if True:
#    con03_stage = scr.merge(con03_base, left_on ='ENCOUNTERID', right_on='ENCOUNTERID', how='left')
    con03_stage = scravg.merge(con03_base_rn1, left_on =['PATID','ONSETS_ENCOUNTERID'], right_on=['PATID','ONSETS_ENCOUNTERID'], how='left')
    con03_stage['DAYS_SINCE_ADMIT_DIFF'] = con03_stage['DAYS_SINCE_ADMIT']-con03_stage['DAYS_SINCE_ADMIT_BASE']

    # Condition 2
    con03_stage = con03_stage[con03_stage['DAYS_SINCE_ADMIT_DIFF']<=7]
    con04_stage = con03_stage.copy()
    con04_stage['RESULT_NUM_INC'] = np.round(con03_stage['RESULT_NUM']/con03_stage['RESULT_NUM_BASE'],1)

    conditions = [
        (con04_stage['RESULT_NUM_INC'] < 1.5),
        (con04_stage['RESULT_NUM_INC'] >= 1.5) & (con04_stage['RESULT_NUM_INC'] < 2.0),
        (con04_stage['RESULT_NUM_INC'] >= 2.0) & (con04_stage['RESULT_NUM_INC'] < 3.0),
        (con04_stage['RESULT_NUM_INC'] >= 3)
        ]
    values = [0, 1, 2, 3]
    con04_stage['AKI_STAGE'] = np.select(conditions, values)

In [ ]:
# CONSORT_DIAGRAM
con05_stage = con04_stage[con04_stage['AKI_STAGE']!=0].copy()
con05_stage = con05_stage.loc[con05_stage[['PATID','ONSETS_ENCOUNTERID','AKI_STAGE', 'DAYS_SINCE_ADMIT_DIFF']].drop_duplicates().groupby(['PATID','ONSETS_ENCOUNTERID','AKI_STAGE']).idxmin().reset_index()['DAYS_SINCE_ADMIT_DIFF']]

In [ ]:
#consort_diagram = con04_stage[['PATID','ONSETS_ENCOUNTERID','AKI_STAGE', 'DAYS_SINCE_ADMIT_DIFF']].drop_duplicates()
con06_stage = pd.pivot(con05_stage, index=['PATID','ONSETS_ENCOUNTERID'], columns='AKI_STAGE', values='DAYS_SINCE_ADMIT_DIFF').reset_index()
con06_stage.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'STAGE_1', 'STAGE_2', 'STAGE_3']
consort_diagram = pd.DataFrame({'ENT_TYP': ['AKI1', 'AKI2', 'AKI3', 'AKI1to2', 'AKI1to2to3', 'AKI2to3'],
                               'ENT_CNT':[con06_stage['STAGE_1'].notnull().sum(), con06_stage['STAGE_2'].notnull().sum(), con06_stage['STAGE_3'].notnull().sum(), 
                                          np.logical_and(con06_stage['STAGE_1'].notnull(), con06_stage['STAGE_2'].notnull()).sum(),
                                          np.logical_and(np.logical_and(con06_stage['STAGE_1'].notnull(), con06_stage['STAGE_2'].notnull()), con06_stage['STAGE_3'].notnull()).sum(),
                                          np.logical_and(con06_stage['STAGE_2'].notnull(), con06_stage['STAGE_3'].notnull()).sum()]})
consort_diagram

In [ ]:
onsetcsv[['ENCOUNTERID']].drop_duplicates().shape

In [ ]:
10000/149682*100

In [ ]:
con07_stage_1 = con05_stage[con05_stage['AKI_STAGE']==1][['PATID', 'ONSETS_ENCOUNTERID', 'SPECIMEN_DATE_TIME', 'DAYS_SINCE_ADMIT', 'RESULT_NUM', 'RESULT_NUM_INC']]
con07_stage_1.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'AKI1_ONSET', 'AKI1_SINCE_ADMIT', 'AKI1_SCR', 'AKI1_INC']
con07_stage_2 = con05_stage[con05_stage['AKI_STAGE']==2][['PATID', 'ONSETS_ENCOUNTERID', 'SPECIMEN_DATE_TIME', 'DAYS_SINCE_ADMIT', 'RESULT_NUM', 'RESULT_NUM_INC']]
con07_stage_2.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'AKI2_ONSET', 'AKI2_SINCE_ADMIT', 'AKI2_SCR', 'AKI2_INC']
con07_stage_3 = con05_stage[con05_stage['AKI_STAGE']==3][['PATID', 'ONSETS_ENCOUNTERID', 'SPECIMEN_DATE_TIME', 'DAYS_SINCE_ADMIT', 'RESULT_NUM', 'RESULT_NUM_INC']]
con07_stage_3.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'AKI3_ONSET', 'AKI3_SINCE_ADMIT', 'AKI3_SCR', 'AKI3_INC']

In [ ]:
if True:
# AKI3 based on rrt
    pxcsv=pd.read_csv(home_dir+site+'/raw/AKI_PX.csv')    
    pxcsv.columns = [y.upper() for y in pxcsv.columns]        
    pxcsv.columns = [x.replace('"+PD.DATE_SHIFT"', '') for x in pxcsv.columns]                
    rrt_stage = pxcsv[np.logical_or(np.logical_or(np.logical_and(pxcsv['PX_TYPE']=='CH',pxcsv['PX'].isin(['50300','50320','50323','50325','50327','50328','50329','50340','50360','50365','50370','50380'])),
                           np.logical_and(pxcsv['PX_TYPE']=='09',pxcsv['PX'].isin(['55.51','55.52','55.53','55.54','55.61','55.69']))),np.logical_or(
                           np.logical_and(pxcsv['PX_TYPE']=='9',pxcsv['PX'].isin(['55.51','55.52','55.53','55.54','55.61','55.69'])),                       
                           np.logical_and(pxcsv['PX_TYPE']=='10',pxcsv['PX'].isin(['0TY00Z0','0TY00Z1','0TY00Z2','0TY10Z0','0TY10Z1','0TY10Z2','0TB00ZZ','0TB10ZZ','0TT00ZZ','0TT10ZZ','0TT20ZZ']))))]
  
    datafolder = home_dir
    con04_stage.to_pickle(home_dir+site+'/p0_akistageslope_'+site+'.pkl')    
    rrt_stage.to_pickle(home_dir+site+'/p0_akistagerrt_'+site+'.pkl')

In [ ]:
rrt_stage = convert_daytime(rrt_stage)
rrt_stage = rrt_stage[['PATID','ONSETS_ENCOUNTERID','PX_DATE']]
rrt_stage.columns = ['PATID','ONSETS_ENCOUNTERID','RRT3_ONSET']

onsetcsv_merge = onsetcsv[['PATID','ENCOUNTERID','ADMIT_DATE']].copy()
onsetcsv_merge.columns = ['PATID','ONSETS_ENCOUNTERID','ADMIT_DATE']

rrt_stage = rrt_stage.merge(onsetcsv_merge, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='left')
rrt_stage['RRT3_SINCE_ADMIT'] = (rrt_stage['RRT3_ONSET']-rrt_stage['ADMIT_DATE']).dt.total_seconds()/3600/24
rrt_stage = rrt_stage.loc[rrt_stage[['ONSETS_ENCOUNTERID', 'RRT3_SINCE_ADMIT']].groupby('ONSETS_ENCOUNTERID').idxmin().reset_index()['RRT3_SINCE_ADMIT']]

In [ ]:
con07_stage_rrt = rrt_stage.copy().drop('ADMIT_DATE',axis=1)
con07_stage_rrt.columns = ['PATID','ONSETS_ENCOUNTERID','AKI3_ONSET','AKI3_SINCE_ADMIT']
con07_stage_rrt['AKI3_SCR'] = np.nan
con07_stage_rrt['AKI3_INC'] = np.nan

con07_stage_3r = pd.concat([con07_stage_3, con07_stage_rrt])
con07_stage_3r = con07_stage_3r.loc[con07_stage_3r[['ONSETS_ENCOUNTERID', 'AKI3_SINCE_ADMIT']].groupby('ONSETS_ENCOUNTERID').idxmin().reset_index()['AKI3_SINCE_ADMIT']]

In [ ]:
con07_stage_0 = con04_stage[con04_stage['AKI_STAGE']==0].copy()
con07_stage_0 = con07_stage_0.loc[con07_stage_0[['PATID','ONSETS_ENCOUNTERID','AKI_STAGE', 'DAYS_SINCE_ADMIT_DIFF']].drop_duplicates().groupby(['PATID','ONSETS_ENCOUNTERID','AKI_STAGE']).idxmax().reset_index()['DAYS_SINCE_ADMIT_DIFF']]

In [ ]:
con07_stage_0 = con07_stage_0[['PATID', 'ONSETS_ENCOUNTERID', 'SPECIMEN_DATE_TIME', 'DAYS_SINCE_ADMIT', 'RESULT_NUM', 'RESULT_NUM_INC']]
con07_stage_0.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'NONAKI_ANCHOR', 'NONAKI_SINCE_ADMIT', 'NON_AKI_SCR', 'NON_AKI_INC']

In [ ]:
#Combine all stages
con07_stage_scr = con03_base_rn1[['PATID', 'ONSETS_ENCOUNTERID', 'RESULT_NUM_BASE']].drop_duplicates()
con07_stage_scr.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'SERUM_CREAT_BASE']
con07_stage = con07_stage_scr.merge(con07_stage_0, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer').merge(con07_stage_1, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer').merge(con07_stage_2, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer').merge(con07_stage_3r, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer')

In [ ]:
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI1_ONSET'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI1_SCR'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI1_INC'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI1_SINCE_ADMIT'] = np.nan

con07_stage.loc[con07_stage['AKI2_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI2_ONSET'] = np.nan
con07_stage.loc[con07_stage['AKI2_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI2_SCR'] = np.nan
con07_stage.loc[con07_stage['AKI2_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI2_INC'] = np.nan
con07_stage.loc[con07_stage['AKI2_SINCE_ADMIT']>=con07_stage['AKI3_SINCE_ADMIT'], 'AKI2_SINCE_ADMIT'] = np.nan

con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI2_SINCE_ADMIT'], 'AKI1_ONSET'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI2_SINCE_ADMIT'], 'AKI1_SCR'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI2_SINCE_ADMIT'], 'AKI1_INC'] = np.nan
con07_stage.loc[con07_stage['AKI1_SINCE_ADMIT']>=con07_stage['AKI2_SINCE_ADMIT'], 'AKI1_SINCE_ADMIT'] = np.nan

In [ ]:
con07_stage.loc[np.logical_or(np.logical_or(con07_stage['AKI1_SINCE_ADMIT'].notnull(),con07_stage['AKI2_SINCE_ADMIT'].notnull()),con07_stage['AKI3_SINCE_ADMIT'].notnull()), 'NONAKI_ANCHOR'] = np.nan
con07_stage.loc[np.logical_or(np.logical_or(con07_stage['AKI1_SINCE_ADMIT'].notnull(),con07_stage['AKI2_SINCE_ADMIT'].notnull()),con07_stage['AKI3_SINCE_ADMIT'].notnull()), 'NONAKI_SINCE_ADMIT'] = np.nan
con07_stage.loc[np.logical_or(np.logical_or(con07_stage['AKI1_SINCE_ADMIT'].notnull(),con07_stage['AKI2_SINCE_ADMIT'].notnull()),con07_stage['AKI3_SINCE_ADMIT'].notnull()), 'NON_AKI_SCR'] = np.nan
con07_stage.loc[np.logical_or(np.logical_or(con07_stage['AKI1_SINCE_ADMIT'].notnull(),con07_stage['AKI2_SINCE_ADMIT'].notnull()),con07_stage['AKI3_SINCE_ADMIT'].notnull()), 'NON_AKI_INC'] = np.nan

In [ ]:
onsetcsv_merge = onsetcsv[['PATID','ENCOUNTERID','ADMIT_DATE','ADMIT_YEAR','DISCHARGE_DATE', 'BCCOVID']].copy()
onsetcsv_merge.columns = ['PATID','ONSETS_ENCOUNTERID','ADMIT_DATE','ADMIT_YEAR','DISCHARGE_DATE', 'BCCOVID']
con07_stage = con07_stage.merge(onsetcsv_merge, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer')

In [ ]:
con07_stagetest.to_csv((home_dir+site+'/raw/AKI_ONSETS_NEW.csv'),index=False)

In [ ]:
onsetcsvtest = onsetcsv.copy()
onsetcsvtest = onsetcsvtest.rename(columns = {'ENCOUNTERID':'ONSETS_ENCOUNTERID'})
con07_stagetest = con07_stage.copy()
con07_stagetest = con07_stagetest.rename(columns = {'ENCOUNTERID':'ONSETS_ENCOUNTERID'})
#onsetcsvtest
testmerge = con07_stagetest.merge(onsetcsvtest, right_on=['PATID', 'ONSETS_ENCOUNTERID'], left_on=['PATID', 'ONSETS_ENCOUNTERID'], how='outer')

In [ ]:
comparecolname = 'AKI2_INC'
comparecolname_eql = 'AKI3_INC'
comparecolname_eql2 = 'AKI3_INC'
#testmerge[testmerge[comparecolname+'_x'] != testmerge[comparecolname+'_y']][['PATID','ONSETS_ENCOUNTERID',comparecolname+'_x',comparecolname+'_y']]
#testmerge2 = testmerge[np.logical_or(np.logical_and(testmerge[comparecolname+'_x'].notnull(), testmerge[comparecolname+'_y'].isnull()), np.logical_and(testmerge[comparecolname+'_x'].isnull(), testmerge[comparecolname+'_y'].notnull()))]#[['PATID','ONSETS_ENCOUNTERID',comparecolname+'_x',comparecolname+'_y']].head(10)
#testmerge3 = testmerge2[np.logical_or(np.logical_and(testmerge[comparecolname_eql+'_x'].notnull(), testmerge[comparecolname_eql+'_y'].isnull()), np.logical_and(testmerge[comparecolname_eql+'_x'].isnull(), testmerge[comparecolname_eql+'_y'].notnull()))]
#testmerge3[np.logical_or(np.logical_and(testmerge[comparecolname_eql2+'_x'].notnull(), testmerge[comparecolname_eql2+'_y'].isnull()), np.logical_and(testmerge[comparecolname_eql2+'_x'].isnull(), testmerge[comparecolname_eql2+'_y'].notnull()))][['PATID','ONSETS_ENCOUNTERID',comparecolname_eql2+'_x',comparecolname_eql2+'_y']].head(10)
#testmerge2[['PATID','ONSETS_ENCOUNTERID',comparecolname+'_x',comparecolname+'_y']].head(10)

testmerge2 = testmerge[np.logical_and(testmerge[comparecolname+'_x'].notnull(), testmerge[comparecolname+'_y'].isnull())][['PATID','ONSETS_ENCOUNTERID',comparecolname+'_x',comparecolname+'_y']]